# Exploratory Data Analysis - Insurance Risk Analytics

This notebook contains exploratory data analysis for the AlphaCare Insurance Solutions (ACIS) project. We'll analyze historical claims data to identify low-risk customer segments for targeted premium reductions.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plotting style
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

## 1. Data Generation

For this notebook, we'll create synthetic data for demonstration purposes.

In [ ]:
# Generate synthetic data
np.random.seed(42)
n_samples = 1000

# Create dummy data
df = pd.DataFrame({
    'PolicyID': range(1, n_samples + 1),
    'Province': np.random.choice(['Western Cape', 'Gauteng', 'KwaZulu-Natal', 'Eastern Cape'], n_samples),
    'Gender': np.random.choice(['M', 'F'], n_samples),
    'VehicleType': np.random.choice(['Sedan', 'SUV', 'Hatchback', 'Truck', 'Sports'], n_samples),
    'VehicleMake': np.random.choice(['Toyota', 'Volkswagen', 'Ford', 'BMW', 'Mercedes'], n_samples),
    'CustomValueEstimate': np.random.normal(200000, 50000, n_samples),
    'TotalPremium': np.random.normal(5000, 1000, n_samples),
    'TotalClaims': np.random.exponential(2000, n_samples) * np.random.binomial(1, 0.2, n_samples),
    'PolicyStartDate': pd.date_range(start='2014-02-01', periods=n_samples)
})

print(f"Created synthetic data. Shape: {df.shape}")

## 2. Data Overview

In [ ]:
# Display basic information
print("Dataset shape:")
print(df.shape)

print("\nColumn names:")
print(df.columns.tolist())

print("\nData types:")
print(df.dtypes)

print("\nMissing values:")
print(df.isnull().sum())

In [ ]:
# Display sample data
df.head()

In [ ]:
# Display summary statistics
df.describe(include='all').T

## 3. Data Preprocessing

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# Calculate loss ratio
df_processed['LossRatio'] = df_processed['TotalClaims'] / df_processed['TotalPremium']

# Replace infinite values with NaN and then with a large value
df_processed['LossRatio'] = df_processed['LossRatio'].replace([float('inf')], float('nan'))
df_processed['LossRatio'] = df_processed['LossRatio'].fillna(0.0)  # Replace NaN with 0

print(f"Processed data shape: {df_processed.shape}")

# Check for derived features
print("\nNew columns added:")
new_cols = set(df_processed.columns) - set(df.columns)
print(new_cols)

## 4. Portfolio Performance Analysis

In [ ]:
# Calculate overall loss ratio
overall_loss_ratio = df_processed['TotalClaims'].sum() / df_processed['TotalPremium'].sum()
print(f"Overall Loss Ratio: {overall_loss_ratio:.4f}")

# Calculate claim frequency
claim_frequency = (df_processed['TotalClaims'] > 0).mean()
print(f"Claim Frequency: {claim_frequency:.4f} ({claim_frequency*100:.2f}%)")

# Calculate average premium and claim
avg_premium = df_processed['TotalPremium'].mean()
avg_claim = df_processed[df_processed['TotalClaims'] > 0]['TotalClaims'].mean()
print(f"Average Premium: {avg_premium:.2f}")
print(f"Average Claim (for policies with claims): {avg_claim:.2f}")

## 5. Segment Analysis

In [ ]:
# Loss ratio by province
province_lr = df_processed.groupby('Province')['LossRatio'].mean().sort_values()

plt.figure(figsize=(12, 8))
ax = province_lr.plot(kind='barh')
plt.title('Loss Ratio by Province')
plt.xlabel('Loss Ratio (Claims / Premium)')
plt.ylabel('Province')
plt.axvline(x=1.0, color='red', linestyle='--', label='Break-even point')
for i, v in enumerate(province_lr):
    ax.text(v + 0.02, i, f'{v:.2f}', va='center')
plt.legend()
plt.tight_layout()

In [ ]:
# Loss ratio by vehicle type
vehicle_lr = df_processed.groupby('VehicleType')['LossRatio'].mean().sort_values()

plt.figure(figsize=(12, 8))
ax = vehicle_lr.plot(kind='barh')
plt.title('Loss Ratio by Vehicle Type')
plt.xlabel('Loss Ratio (Claims / Premium)')
plt.ylabel('Vehicle Type')
plt.axvline(x=1.0, color='red', linestyle='--', label='Break-even point')
for i, v in enumerate(vehicle_lr):
    ax.text(v + 0.02, i, f'{v:.2f}', va='center')
plt.legend()
plt.tight_layout()

In [ ]:
# Loss ratio by gender
gender_lr = df_processed.groupby('Gender')['LossRatio'].mean().sort_values()

plt.figure(figsize=(12, 8))
ax = gender_lr.plot(kind='barh')
plt.title('Loss Ratio by Gender')
plt.xlabel('Loss Ratio (Claims / Premium)')
plt.ylabel('Gender')
plt.axvline(x=1.0, color='red', linestyle='--', label='Break-even point')
for i, v in enumerate(gender_lr):
    ax.text(v + 0.02, i, f'{v:.2f}', va='center')
plt.legend()
plt.tight_layout()

## 6. Premium vs Claims Analysis

In [ ]:
# Scatter plot of premium vs claims
plt.figure(figsize=(12, 8))
ax = sns.scatterplot(data=df_processed, x='TotalPremium', y='TotalClaims', hue='Province', alpha=0.7)
max_val = max(df_processed['TotalPremium'].max(), df_processed['TotalClaims'].max())
plt.plot([0, max_val], [0, max_val], 'r--', label='Premium = Claims')
plt.title('Total Claims vs Total Premium')
plt.xlabel('Total Premium')
plt.ylabel('Total Claims')
plt.legend()
plt.tight_layout()

## 7. Claim Distribution Analysis

In [ ]:
# Distribution of claim amounts
plt.figure(figsize=(12, 8))
claims_df = df_processed[df_processed['TotalClaims'] > 0]
ax = sns.histplot(data=claims_df, x='TotalClaims', kde=True)
mean_claim = claims_df['TotalClaims'].mean()
plt.axvline(mean_claim, color='red', linestyle='--', label=f'Mean: {mean_claim:.2f}')
median_claim = claims_df['TotalClaims'].median()
plt.axvline(median_claim, color='green', linestyle='--', label=f'Median: {median_claim:.2f}')
plt.title('Distribution of Claim Amounts')
plt.xlabel('Total Claims')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()

## 8. Vehicle Analysis

In [ ]:
# Analyze vehicle makes by risk
vehicle_risk = df_processed.groupby('VehicleMake').agg({
    'TotalPremium': 'sum',
    'TotalClaims': 'sum',
    'PolicyID': 'count'
}).rename(columns={'PolicyID': 'PolicyCount'})

vehicle_risk['LossRatio'] = vehicle_risk['TotalClaims'] / vehicle_risk['TotalPremium']
vehicle_risk['AvgPremium'] = vehicle_risk['TotalPremium'] / vehicle_risk['PolicyCount']
vehicle_risk['AvgClaim'] = vehicle_risk['TotalClaims'] / vehicle_risk['PolicyCount']

# Sort by loss ratio
vehicle_risk_sorted = vehicle_risk.sort_values('LossRatio', ascending=False)
vehicle_risk_sorted

In [ ]:
# Plot vehicle makes by loss ratio
plt.figure(figsize=(12, 8))
sns.barplot(x=vehicle_risk_sorted.index, y='LossRatio', data=vehicle_risk_sorted)
plt.axhline(y=1.0, color='red', linestyle='--', label='Break-even point')
plt.title('Loss Ratio by Vehicle Make')
plt.xlabel('Vehicle Make')
plt.ylabel('Loss Ratio')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()

## 9. Key Insights and Findings

In [ ]:
# Overall portfolio performance
overall_loss_ratio = df_processed['TotalClaims'].sum() / df_processed['TotalPremium'].sum()
claim_frequency = (df_processed['TotalClaims'] > 0).mean()

print("1. Overall portfolio performance:")
print(f"   - Overall loss ratio: {overall_loss_ratio:.4f}")
print(f"   - Claim frequency: {claim_frequency:.4f} ({claim_frequency*100:.2f}%)")

# Segment insights
province_lr = df_processed.groupby('Province')['LossRatio'].mean().sort_values()
vehicle_lr = df_processed.groupby('VehicleType')['LossRatio'].mean().sort_values()
gender_lr = df_processed.groupby('Gender')['LossRatio'].mean().sort_values()

print("\n2. Segment insights:")
print(f"   - Provinces with lowest risk: {', '.join(province_lr.index[:2])}")
print(f"   - Vehicle types with lowest risk: {', '.join(vehicle_lr.index[:2])}")
print(f"   - Gender-based risk differences: {gender_lr.index[0]} has lower risk than {gender_lr.index[-1]}")

# Vehicle insights
vehicle_risk = df_processed.groupby('VehicleMake').agg({
    'TotalPremium': 'sum',
    'TotalClaims': 'sum'
})
vehicle_risk['LossRatio'] = vehicle_risk['TotalClaims'] / vehicle_risk['TotalPremium']
vehicle_risk_sorted = vehicle_risk.sort_values('LossRatio')

print("\n3. Vehicle insights:")
print(f"   - Makes with lowest risk: {', '.join(vehicle_risk_sorted.index[:2])}")
print(f"   - Makes with highest risk: {', '.join(vehicle_risk_sorted.index[-2:])}")

## 10. Recommendations for Further Analysis

Based on our findings, we recommend the following areas for further investigation:

1. **Interaction Effects**: Analyze how province and vehicle type interact to affect risk
2. **Time-Based Patterns**: Investigate if there are seasonal or temporal patterns in claims
3. **Customer Demographics**: Explore additional demographic factors if available
4. **Claim Frequency vs. Severity**: Analyze these two components separately
5. **Premium Optimization**: Develop a model to optimize premiums based on risk factors